#### LeetCode - SQL - #180

Refer [180. Consecutive Numbers](https://leetcode.com/problems/consecutive-numbers/description/)

Find all numbers that appear at least three times consecutively.

Return the result table in  **any order**.

In [0]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
columns = ['id', 'num']
schema = {'id':'Int64', 'num':'Int64'}

In [0]:
import pandas

numbers_pandas_dataframe = pandas.DataFrame(data=data, columns=columns).astype(schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
numbers_dataframe = spark_context.createDataFrame(numbers_pandas_dataframe)
numbers_dataframe.printSchema()
numbers_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- num: long (nullable = true)

+---+---+
| id|num|
+---+---+
|  1|  1|
|  2|  1|
|  3|  1|
|  4|  2|
|  5|  1|
+---+---+
only showing top 5 rows



#### Using Window Functions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, lead

window_spec = Window.orderBy(numbers_dataframe.id)
numbers_dataframe = \
    numbers_dataframe \
        .withColumn("lag_num", lag(numbers_dataframe.num, 1).over(window_spec)) \
        .withColumn("lead_num", lead(numbers_dataframe.num, 1).over(window_spec))
numbers_dataframe.show(5)

numbers_dataframe \
    .filter((numbers_dataframe.num == numbers_dataframe.lag_num) & (numbers_dataframe.num == numbers_dataframe.lead_num)) \
    .select(numbers_dataframe.num.alias("ConsecutiveNums")) \
    .show()

+---+---+-------+--------+
| id|num|lag_num|lead_num|
+---+---+-------+--------+
|  1|  1|   NULL|       1|
|  2|  1|      1|       1|
|  3|  1|      1|       2|
|  4|  2|      1|       1|
|  5|  1|      2|       2|
+---+---+-------+--------+
only showing top 5 rows

+---------------+
|ConsecutiveNums|
+---------------+
|              1|
+---------------+



#### Using SQL Joins

In [0]:
# from pyspark.sql.functions import col

# root_dataframe = numbers_dataframe.alias("root_dataframe")
# lead_dataframe = numbers_dataframe.alias("lead_dataframe")

# joined_df = \
#     numbers_dataframe.alias("root") \
#         .join(numbers_dataframe.alias("a"), (col("root.num") + 1) == col("a.num"), "inner") \
#         .select(col("root.num"), col("a.num"))
#         .join(numbers_dataframe.alias("second"), (col("first.num") + 1) == col("second.num"), "inner")
# joined_df.show()
# numbers_dataframe.as("root") \
#     .join(lead_dataframe.as("lead"), "lead.num" == ("root.num" - 1), "inner") \
#     .join(lag_dataframe.as("lag"), ("root.num" + 1) == "lag.num", "inner") \
#     .filter(lead_dataframe.num == numbers_dataframe.num & numbers_dataframe.num == lag_dataframe.num) \
#     .select(numbers_dataframe.num.alias("ConsecutiveNums")) \
#     .show()

+---+---+
|num|num|
+---+---+
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
|  1|  2|
+---+---+

